# Extracting GML Coordinates

In [48]:
import requests
import xml.etree.ElementTree as ET

In [49]:
productID = "864175b7-759d-4ccc-a878-c97d6aa11275"
r = requests.get("http://s5phub.copernicus.eu/dhus/odata/v1/Products("+"'"+productID+"'"+")", auth=('s5pguest', 's5pguest'))
root = ET.fromstring(r.content)

In [50]:
geo = root.find("{http://schemas.microsoft.com/ado/2007/08/dataservices/metadata}properties")

In [51]:
properties = {}

for child in geo:
    if((child.tag[55:] == "ContentDate") or (child.tag[55:] == "Checksum")):
            for subchild in geo.find(child.tag):
                if(child.tag[55:] == "ContentDate"):
                    properties["ContentDate_"+subchild.tag[55:]] = geo.find(child.tag).find(subchild.tag).text
                else:
                    properties["Checksum_"+subchild.tag[55:]] = geo.find(child.tag).find(subchild.tag).text
    else:
        if((child.tag[55:] != "ContentGeometry")):
            properties[child.tag[55:]] = geo.find(child.tag).text

In [52]:
for k in properties.keys():
    print(str(k)+" "+str(properties[k]))

Id 864175b7-759d-4ccc-a878-c97d6aa11275
Name S5P_OFFL_L2__NO2____20181009T213513_20181009T231642_05129_01_010100_20181015T231412
ContentType application/octet-stream
ContentLength 331745241
ChildrenNumber 0
Value None
CreationDate 2018-10-21T18:17:01.281
IngestionDate 2018-10-21T18:16:29.745
EvictionDate 2018-10-21T18:17:01.646
ContentDate_Start 2018-10-09T21:56:46.000
ContentDate_End 2018-10-09T22:55:11.000
Checksum_Algorithm MD5
Checksum_Value 746366C922CC64A2DD85FB927EE70A70


In [53]:
ContentGeometry = list(geo)[len(list(geo))-1]

In [54]:
gml = ET.fromstring(ContentGeometry.text)

In [55]:
EPSG = gml.get('srsName')[40:]
print(EPSG)

4326


In [56]:
TYPE  = gml.tag[28:]
print(TYPE)

Polygon


In [57]:
outerBoundaryIs = list(gml)[0]
LinearRingin = list(outerBoundaryIs)[0]
COORDINATES = list(LinearRingin)[0]

In [58]:
preproc = COORDINATES.text.split(",")

In [60]:
len(preproc)

266

In [15]:
for i in range(len(preproc)):
    if(preproc[i] in preproc[i+1:]):
        print("S")

In [16]:
p = preproc[0:len(preproc)-1]

In [17]:
p[0] = preproc[len(preproc)-2]

In [18]:
geoJsonCoordinates = []
for temp in p:
    Lat,Leng = temp.split(' ')
    geoJsonCoordinates.append([float(Lat),float(Leng)])

In [19]:
import json

In [20]:
geometry = {}
geometry['type'] = 'Polygon'
geometry['coordinates'] = [geoJsonCoordinates]

In [21]:
feature = {}
feature['type'] = "Feature"
feature['properties'] = {}
feature['geometry'] = geometry

In [22]:
FeatureCollection = {}
FeatureCollection['type'] = "FeatureCollection"
FeatureCollection['features'] = [feature]

In [23]:
json_data = json.dumps(FeatureCollection)

In [27]:
json_data

'{"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[104.37874, 66.93215], [104.37874, 67.65671], [109.04739, 68.23474], [113.98591, 68.65444], [119.14787, 68.90659], [124.46878, 68.98494], [129.8712, 68.88757], [135.2687, 68.616875], [140.57614, 68.1795], [145.71619, 67.585], [150.62692, 66.84553], [155.2643, 65.974266], [159.6013, 64.98531], [163.6284, 63.891895], [167.34741, 62.70634], [170.76971, 61.440132], [173.9122, 60.103695], [176.79602, 58.70588], [179.44254, 57.25446], [-178.1269, 55.75615], [-175.89157, 54.217136], [-173.83156, 52.641926], [-171.92982, 51.035313], [-170.16974, 49.400864], [-168.53711, 47.741756], [-167.01912, 46.06094], [-165.60406, 44.360435], [-164.2822, 42.64255], [-163.04431, 40.909245], [-161.88228, 39.16187], [-160.7892, 37.402092], [-159.75845, 35.630833], [-158.78491, 33.849392], [-157.86351, 32.05879], [-156.98932, 30.259676], [-156.15918, 28.452955], [-155.369, 26.63931

In [26]:
#mod1 = ContentGeometry.text.replace("\n","")
#mod1 = mod1.replace("&lt;","<")
#mod1 = mod1.replace("&gt;",">")

# Downloading nc file

In [3]:
f = requests.get("http://s5phub.copernicus.eu/dhus/odata/v1/Products("+"'"+productID+"'"+")/$value", auth=('s5pguest', 's5pguest'))

In [4]:
with open('temp.nc', 'wb') as z:
    z.write(f.content)

In [5]:
from netCDF4 import Dataset
nc=Dataset('temp.nc','r')

In [10]:
for k in nc.ncattrs():
    print(k,nc.getncattr(k))

Conventions CF-1.7
institution KNMI
source Sentinel 5 precursor, TROPOMI, space-borne remote sensing, L2
summary TROPOMI/S5P NO2 1-Orbit L2 Swath 7x3.5km
tracking_id 95bdb98e-073f-4a32-80e4-d6c53e444108
id S5P_OFFL_L2__NO2____20181009T213513_20181009T231642_05129_01_010100_20181015T231412
time_reference 2018-10-09T00:00:00Z
time_reference_days_since_1950 25118
time_reference_julian_day 2458400.5
time_reference_seconds_since_1970 1539043200
time_coverage_start 2018-10-09T21:56:46Z
time_coverage_end 2018-10-09T22:55:11Z
time_coverage_duration PT3505.000S
time_coverage_resolution PT1.080S
orbit 5129
references http://www.tropomi.eu/data-products/nitrogen-dioxide
processor_version 1.1.0
keywords_vocabulary AGU index terms, http://publications.agu.org/author-resource-center/index-terms/
keywords 0345 Pollution, Urban and Regional; 0365 Troposphere, Composition and Chemistry; 0368 Troposphere, Constituent Transport and Chemistry; 3360 Remote Sensing; 3363 Stratospheric Dynamics
standard_name

In [18]:
g = dict(nc.groups)

In [17]:
for v in g['PRODUCT'].variables:
    print(v)

scanline
ground_pixel
time
corner
polynomial_exponents
intensity_offset_polynomial_exponents
layer
vertices
latitude
longitude
delta_time
time_utc
qa_value
nitrogendioxide_tropospheric_column
nitrogendioxide_tropospheric_column_precision
nitrogendioxide_tropospheric_column_precision_kernel
averaging_kernel
air_mass_factor_troposphere
air_mass_factor_total
tm5_tropopause_layer_index
tm5_constant_a
tm5_constant_b


In [19]:
ncv = g['PRODUCT'].variables
ncv.keys()

odict_keys(['scanline', 'ground_pixel', 'time', 'corner', 'polynomial_exponents', 'intensity_offset_polynomial_exponents', 'layer', 'vertices', 'latitude', 'longitude', 'delta_time', 'time_utc', 'qa_value', 'nitrogendioxide_tropospheric_column', 'nitrogendioxide_tropospheric_column_precision', 'nitrogendioxide_tropospheric_column_precision_kernel', 'averaging_kernel', 'air_mass_factor_troposphere', 'air_mass_factor_total', 'tm5_tropopause_layer_index', 'tm5_constant_a', 'tm5_constant_b'])

In [45]:
len(ncv['longitude'][0].data)

3246

In [46]:
len(ncv['latitude'][0].data)

3246

In [20]:
lon = ncv['longitude'][10:-10:2,20:-10:2]
lat = ncv['latitude'][10:-10:2,20:-10:2]

In [21]:
# read the 1st time step
itime = 0
nitrogendioxide = ncv['nitrogendioxide_tropospheric_column'][itime,10:-10:2,20:-10:2]